# Imports

In [1]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split

In [5]:
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
import torch.nn as nn
from transformers import BertModel, BertTokenizer
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import requests

In [6]:
urls = [
    "https://raw.githubusercontent.com/kzGarifullin/TDA-for-Transformers/main/scripts/directed_TDA_train.py",
    "https://raw.githubusercontent.com/kzGarifullin/TDA-for-Transformers/main/scripts/baseline_train.py"
]

for url in urls:
    response = requests.get(url)
    filename = url.split("/")[-1]
    with open(filename, 'wb') as file:
        file.write(response.content)

print("Files downloaded successfully.")


Files downloaded successfully.


In [7]:
from directed_TDA_train import directed_epoch_validation, Directed_classifier, train_directed
from baseline_train import baseline_epoch_validation, train_baseline, Classifier

# Dataset preparation

In [8]:
imdb_dataset = load_dataset("imdb")
texts = imdb_dataset["train"]["text"]
labels = imdb_dataset["train"]["label"]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [10]:

urls = [
    "https://raw.githubusercontent.com/kzGarifullin/TDA-for-Transformers/main/features/directed_features_1layer_1head.npy",
    "https://raw.githubusercontent.com/kzGarifullin/TDA-for-Transformers/main/features/undirected_features_1layer_1head.npy"
]

for url in urls:
    response = requests.get(url)
    filename = url.split("/")[-1]
    with open(filename, 'wb') as file:
        file.write(response.content)

print("Files downloaded successfully.")

Files downloaded successfully.


In [11]:
undirected_features = np.load('/content/undirected_features_1layer_1head.npy')
undirected_features[undirected_features == np.inf] = 1000
print(undirected_features.shape)
undirected_features_without_nan = undirected_features[:, ~np.isnan(undirected_features).any(axis=0)]
print(undirected_features_without_nan.shape)

(25000, 14)
(25000, 12)


In [12]:

directed_features = np.load('/content/directed_features_1layer_1head.npy')
directed_features[directed_features == np.inf] = 1000
print(directed_features.shape)
directed_features_without_nan = directed_features[:, ~np.isnan(directed_features).any(axis=0)]
print(directed_features_without_nan.shape)


(25000, 14)
(25000, 14)


In [13]:
X_train, X_test, dir_feat_train, dir_feat_test, undir_feat_train, undir_feat_test, y_train, y_test = train_test_split(texts, directed_features_without_nan, undirected_features_without_nan, labels, test_size=0.2, random_state=42)
print(len(X_train))
print(len(X_test))
print(len(dir_feat_train))
print(len(dir_feat_test))
print(len(undir_feat_train))
print(len(undir_feat_test))
print(len(y_train))
print(len(y_test))

20000
5000
20000
5000
20000
5000
20000
5000


In [ ]:
unique_labels, label_counts = np.unique(y_train, return_counts=True)

# Print the label frequencies
for label, count in zip(unique_labels, label_counts):
    print(f"Label {label}: {count} samples")

Label 0: 9985 samples
Label 1: 10015 samples


In [ ]:
unique_labels, label_counts = np.unique(y_test, return_counts=True)

# Print the label frequencies
for label, count in zip(unique_labels, label_counts):
    print(f"Label {label}: {count} samples")

Label 0: 2515 samples
Label 1: 2485 samples


In [15]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_length = 64  # Maximum sequence length for BERT
batch_size = 32
train_token_ids = [tokenizer.encode(text, max_length=max_length, truncation=True) for text in X_train]
val_token_ids = [tokenizer.encode(text, max_length=max_length, truncation=True) for text in X_test]

train_token_ids = [ids + [0] * (max_length - len(ids)) for ids in train_token_ids]
val_token_ids = [ids + [0] * (max_length - len(ids)) for ids in val_token_ids]

train_token_ids_tensor = torch.tensor(train_token_ids)
val_token_ids_tensor = torch.tensor(val_token_ids)
train_labels_tensor = torch.tensor(y_train)
val_labels_tensor = torch.tensor(y_test)

train_dataset = TensorDataset(train_token_ids_tensor, train_labels_tensor)
val_dataset = TensorDataset(val_token_ids_tensor, val_labels_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

# Baseline

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BertModel.from_pretrained('bert-base-uncased')
classifier = Classifier(input_size=768, num_classes=2)
classifier = classifier.to(device)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-3)
num_epochs = 15
print("device:", device)
train_baseline(model, classifier, criterion, optimizer, train_loader, val_loader, num_epochs, device)

device: cuda


100%|██████████| 625/625 [01:17<00:00,  8.09it/s]


Epoch 1/15, Validation Accuracy: 0.6932
Validation MCC: 0.3891122148691857
Train loss:  400.7823181152344
_______________________


100%|██████████| 625/625 [01:17<00:00,  8.09it/s]


Epoch 2/15, Validation Accuracy: 0.711
Validation MCC: 0.42417509923559077
Train loss:  370.2239685058594
_______________________


...

Epoch 13/15, Validation Accuracy: 0.7338
Validation MCC: 0.467577173204401
Train loss:  338.1298522949219
_______________________


100%|██████████| 625/625 [01:17<00:00,  8.06it/s]


Epoch 14/15, Validation Accuracy: 0.733
Validation MCC: 0.46597334921162514
Train loss:  337.4102783203125
_______________________


100%|██████████| 625/625 [01:17<00:00,  8.05it/s]


Epoch 15/15, Validation Accuracy: 0.735
Validation MCC: 0.46997155283678327
Train loss:  336.75860595703125
_______________________


In [ ]:
num_epochs = 5
print("device:", device)
train_baseline(model, classifier, criterion, optimizer, train_loader, val_loader, num_epochs, device)

device: cuda


100%|██████████| 625/625 [01:17<00:00,  8.09it/s]


Epoch 1/5, Validation Accuracy: 0.736
Validation MCC: 0.47197075011749523
Train loss:  336.1638488769531
_______________________


100%|██████████| 625/625 [01:17<00:00,  8.10it/s]


Epoch 2/5, Validation Accuracy: 0.737
Validation MCC: 0.4739705601067291
Train loss:  335.6178283691406
_______________________


...

Epoch 3/5, Validation Accuracy: 0.7366
Validation MCC: 0.47317099483588354
Train loss:  335.1138610839844
_______________________


100%|██████████| 625/625 [01:17<00:00,  8.11it/s]


Epoch 4/5, Validation Accuracy: 0.7374
Validation MCC: 0.47477282362972617
Train loss:  334.6455993652344
_______________________


100%|██████████| 625/625 [01:17<00:00,  8.10it/s]


Epoch 5/5, Validation Accuracy: 0.7386
Validation MCC: 0.477177757662083
Train loss:  334.20953369140625
_______________________


# UnDirected TDA

In [17]:
feat_train_tensor = torch.tensor(undir_feat_train, dtype=torch.float32)
feat_test_tensor = torch.tensor(undir_feat_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)  # Adding a dimension for compatibility
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)  # Adding a dimension for compatibility

feat_train_dataset = TensorDataset(feat_train_tensor, y_train_tensor)
feat_test_dataset = TensorDataset(feat_test_tensor, y_test_tensor)

feat_train_loader = DataLoader(feat_train_dataset, batch_size=batch_size, shuffle=False)
feat_test_loader = DataLoader(feat_test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
undir_classifier = Directed_classifier(input_size=780, num_classes=2)
undir_classifier = undir_classifier.to(device)
model = BertModel.from_pretrained('bert-base-uncased')
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(undir_classifier.parameters(), lr=1e-3)
num_epochs = 15
print("device: ", device)
train_directed(model, optimizer, criterion, undir_classifier, train_loader, val_loader, feat_train_loader, feat_test_loader, num_epochs, device)


device:  cuda


625it [01:17,  8.05it/s]
157it [00:19,  8.08it/s]


Epoch 1/15, Validation Accuracy: 0.6738
Validation MCC: 0.3549454732638499
Train loss:  422.12811279296875
_______________________


625it [01:17,  8.05it/s]
157it [00:19,  8.16it/s]


Epoch 2/15, Validation Accuracy: 0.6938
Validation MCC: 0.390272934388284
Train loss:  384.00146484375
_______________________


...

Epoch 13/15, Validation Accuracy: 0.731
Validation MCC: 0.462899014274819
Train loss:  349.9696044921875
_______________________


625it [01:17,  8.05it/s]
157it [00:19,  8.12it/s]


Epoch 14/15, Validation Accuracy: 0.731
Validation MCC: 0.462899014274819
Train loss:  349.03692626953125
_______________________


625it [01:17,  8.06it/s]
157it [00:19,  8.15it/s]

Epoch 15/15, Validation Accuracy: 0.7298
Validation MCC: 0.4604959170071873
Train loss:  348.37640380859375
_______________________


In [ ]:
num_epochs = 5
print("device: ", device)
train_directed(model, optimizer, criterion, undir_classifier, train_loader, val_loader, feat_train_loader, feat_test_loader, num_epochs, device)


device:  cuda


625it [01:17,  8.08it/s]
157it [00:19,  8.17it/s]


Epoch 1/5, Validation Accuracy: 0.7306
Validation MCC: 0.4620288712017857
Train loss:  347.7237548828125
_______________________


625it [01:17,  8.08it/s]
157it [00:19,  8.13it/s]


Epoch 2/5, Validation Accuracy: 0.7308
Validation MCC: 0.46241807542885127
Train loss:  347.2379150390625
_______________________


...

Epoch 3/5, Validation Accuracy: 0.7328
Validation MCC: 0.4664226867475748
Train loss:  347.3065185546875
_______________________


625it [01:17,  8.09it/s]
157it [00:19,  8.17it/s]


Epoch 4/5, Validation Accuracy: 0.7328
Validation MCC: 0.466400230281764
Train loss:  346.86846923828125
_______________________


625it [01:17,  8.09it/s]
157it [00:19,  8.15it/s]

Epoch 5/5, Validation Accuracy: 0.7336
Validation MCC: 0.46797979962054714
Train loss:  346.55572509765625
_______________________


In [ ]:
num_epochs = 5
print("device: ", device)
train_directed(model, optimizer, criterion, undir_classifier, train_loader, val_loader, feat_train_loader, feat_test_loader, num_epochs, device)


device:  cuda


625it [01:18,  8.00it/s]
157it [00:19,  8.24it/s]


Epoch 1/5, Validation Accuracy: 0.7334
Validation MCC: 0.4675683820156915
Train loss:  346.80169677734375
_______________________


625it [01:17,  8.06it/s]
157it [00:19,  8.15it/s]


Epoch 2/5, Validation Accuracy: 0.7328
Validation MCC: 0.466334669339297
Train loss:  346.35528564453125
_______________________


...

Epoch 3/5, Validation Accuracy: 0.7336
Validation MCC: 0.46797979962054714
Train loss:  345.29669189453125
_______________________


625it [01:17,  8.11it/s]
157it [00:19,  8.14it/s]


Epoch 4/5, Validation Accuracy: 0.7334
Validation MCC: 0.4675683820156915
Train loss:  345.25048828125
_______________________


625it [01:17,  8.05it/s]
157it [00:19,  8.17it/s]

Epoch 5/5, Validation Accuracy: 0.7338
Validation MCC: 0.46836922947444865
Train loss:  344.2669982910156
_______________________


# Directed

In [19]:
feat_train_tensor = torch.tensor(dir_feat_train, dtype=torch.float32)
feat_test_tensor = torch.tensor(dir_feat_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)  # Adding a dimension for compatibility
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)  # Adding a dimension for compatibility

feat_train_dataset = TensorDataset(feat_train_tensor, y_train_tensor)
feat_test_dataset = TensorDataset(feat_test_tensor, y_test_tensor)

feat_train_loader = DataLoader(feat_train_dataset, batch_size=batch_size, shuffle=False)
feat_test_loader = DataLoader(feat_test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dir_classifier = Directed_classifier(input_size=782, num_classes=2)
dir_classifier = dir_classifier.to(device)
model = BertModel.from_pretrained('bert-base-uncased')
model = model.to(device)
criterion = nn.CrossEntropyLoss()
dir_optimizer = torch.optim.Adam(dir_classifier.parameters(), lr=1e-3)
num_epochs = 3
print("device: ", device)
train_directed(model, dir_optimizer, criterion, dir_classifier, train_loader, val_loader, feat_train_loader, feat_test_loader, num_epochs, device)


device:  cuda


5it [00:00,  8.17it/s]We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
625it [01:08,  9.17it/s]
157it [00:17,  8.81it/s]


Epoch 1/3, Validation Accuracy: 0.682
Validation MCC: 0.36439226811110886
Train loss:  411.2994079589844
_______________________


625it [01:13,  8.51it/s]
157it [00:18,  8.26it/s]


Epoch 2/3, Validation Accuracy: 0.7074
Validation MCC: 0.41518176141807117
Train loss:  378.9949951171875
_______________________


625it [01:16,  8.17it/s]
157it [00:19,  8.04it/s]


Epoch 3/3, Validation Accuracy: 0.7132
Validation MCC: 0.42701745203532576
Train loss:  368.2168884277344
_______________________


In [ ]:
num_epochs = 12
print("device: ", device)
train_directed(model, dir_optimizer, criterion, dir_classifier, train_loader, val_loader, feat_train_loader, feat_test_loader, num_epochs, device)


device:  cuda


625it [01:17,  8.06it/s]
157it [00:19,  8.15it/s]


Epoch 1/12, Validation Accuracy: 0.717
Validation MCC: 0.434840635720343
Train loss:  362.3303527832031
_______________________


625it [01:17,  8.09it/s]
157it [00:19,  8.16it/s]


Epoch 2/12, Validation Accuracy: 0.721
Validation MCC: 0.4429903316963208
Train loss:  358.4971008300781
_______________________


...

Epoch 10/12, Validation Accuracy: 0.7268
Validation MCC: 0.4560751684196973
Train loss:  346.70196533203125
_______________________


625it [01:17,  8.06it/s]
157it [00:19,  8.12it/s]


Epoch 11/12, Validation Accuracy: 0.7256
Validation MCC: 0.45386048987859623
Train loss:  345.99261474609375
_______________________


625it [01:17,  8.05it/s]
157it [00:19,  8.13it/s]

Epoch 12/12, Validation Accuracy: 0.7268
Validation MCC: 0.456434166208408
Train loss:  345.3502502441406
_______________________


In [ ]:
num_epochs = 5
print("device: ", device)
train_directed(model, dir_optimizer, criterion, dir_classifier, train_loader, val_loader, feat_train_loader, feat_test_loader, num_epochs, device)


device:  cuda


625it [01:17,  8.04it/s]
157it [00:19,  8.12it/s]


Epoch 1/5, Validation Accuracy: 0.7266
Validation MCC: 0.45626353080951026
Train loss:  344.7642517089844
_______________________


625it [01:17,  8.06it/s]
157it [00:19,  8.13it/s]


Epoch 2/5, Validation Accuracy: 0.7276
Validation MCC: 0.4583829332225241
Train loss:  344.2259521484375
_______________________


...

Epoch 3/5, Validation Accuracy: 0.7274
Validation MCC: 0.45800263336281777
Train loss:  343.7289733886719
_______________________


625it [01:17,  8.09it/s]
157it [00:19,  8.15it/s]


Epoch 4/5, Validation Accuracy: 0.7284
Validation MCC: 0.4600804533616616
Train loss:  343.267822265625
_______________________


625it [01:17,  8.02it/s]
157it [00:19,  8.07it/s]

Epoch 5/5, Validation Accuracy: 0.7296
Validation MCC: 0.4624943025156242
Train loss:  342.83770751953125
_______________________
